# 预处理数据

## 预处理数据集

参考文章：https://zhuanlan.zhihu.com/p/34068451

### 异常数据处理

In [7]:
# imagenet上的猫和狗的类别
dogs_categories = [
 'n02085620','n02085782','n02085936','n02086079','n02086240','n02086646','n02086910','n02087046'
,'n02087394','n02088094','n02088238','n02088364','n02088466','n02088632','n02089078','n02089867'
,'n02089973','n02090379','n02090622','n02090721','n02091032','n02091134','n02091244','n02091467'
,'n02091635','n02091831','n02092002','n02092339','n02093256','n02093428','n02093647','n02093754'
,'n02093859','n02093991','n02094114','n02094258','n02094433','n02095314','n02095570','n02095889'
,'n02096051','n02096177','n02096294','n02096437','n02096585','n02097047','n02097130','n02097209'
,'n02097298','n02097474','n02097658','n02098105','n02098286','n02098413','n02099267','n02099429'
,'n02099601','n02099712','n02099849','n02100236','n02100583','n02100735','n02100877','n02101006'
,'n02101388','n02101556','n02102040','n02102177','n02102318','n02102480','n02102973','n02104029'
,'n02104365','n02105056','n02105162','n02105251','n02105412','n02105505','n02105641','n02105855'
,'n02106030','n02106166','n02106382','n02106550','n02106662','n02107142','n02107312','n02107574'
,'n02107683','n02107908','n02108000','n02108089','n02108422','n02108551','n02108915','n02109047'
,'n02109525','n02109961','n02110063','n02110185','n02110341','n02110627','n02110806','n02110958'
,'n02111129','n02111277','n02111500','n02111889','n02112018','n02112137','n02112350','n02112706'
,'n02113023','n02113186','n02113624','n02113712','n02113799','n02113978']

cats_categories = ['n02123045','n02123159','n02123394','n02123597','n02124075','n02125311','n02127052']

# top-N
top_num = 50

# 数据集路径
# dataset_path = './data/'
# 测试数据位置
dataset_path = './test_data/'

In [2]:
from keras.preprocessing import image
import numpy as np
import os

D:\ProgramData\Anaconda2\envs\keras\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
D:\ProgramData\Anaconda2\envs\keras\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
# 判断是否是猫狗中的一种
def is_cat_or_dog(preds):
    for pred in preds:
        if pred[0] in (dogs_categories+cats_categories):
            return True
    return False

In [ ]:


# 排除异常值
# model_name --- 模型名称：xception, inception_resnet_v2, densenet201
# top_num --- top-N
# img_dir --- 图片存放路径
def exclude_outliers(model_name='xception', top_num=5, img_dir='train/'):
    # import models
    if model_name == 'xception':
        from keras.applications.xception import Xception, preprocess_input, decode_predictions
        model = Xception(weights='imagenet')
        img_size = (299, 299)
    elif model_name == 'inception_resnet_v2':
        from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input, decode_predictions
        model = InceptionResNetV2(weights='imagenet')
        img_size = (299, 299)
    elif model_name == 'densenet201':
        from keras.applications.densenet import DenseNet201, preprocess_input, decode_predictions
        model = DenseNet201(weights='imagenet')
        img_size = (224, 224)

    img_dir = dataset_path + img_dir
    img_list = os.listdir(img_dir)
    outliers = []
    for img_name in img_list:
        if img_name == 'Thumbs.db':
            continue
        img_path = img_dir + img_name
        img = image.load_img(img_path, target_size=img_size) #os.readlink(img_path)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        pred = model.predict(x)
        pred = decode_predictions(pred, top=top_num)[0]
        if not is_cat_or_dog(pred):
            outliers.append(img_name)
    return outliers


#     通过generator实现
#     img_dir = dataset_path + 'some_samples/'
#     gen = image.ImageDataGenerator()
#     img_generator = gen.flow_from_directory(img_dir, target_size=img_size, shuffle=False)
#     print(img_generator)
#     preds = model.predict_generator(img_generator, verbose=1)
    
#     decode_preds = decode_predictions(preds, top=top_num)
#     outliers = []

#     for pred, filename in zip(decode_preds, img_generator.filenames):
#         print(filename)
#         print(pred)
#         print(is_cat_or_dog(pred))
#         if not is_cat_or_dog(pred):
#             outliers.append(filename)
#     return outliers

### 创建outliers文件夹

In [5]:
os.makedirs(os.path.normpath(os.path.join(dataset_path, '../outliers')))

### 利用xception网络识别图片

In [35]:
def exclude_outliers_xception(top_num=top_num, img_dir='train/' ):
    from keras.applications.xception import Xception, preprocess_input, decode_predictions
    model = Xception(weights='imagenet')
    img_size = (299, 299)
    
    img_dir = os.path.normpath(os.path.join(dataset_path, img_dir))
    img_list = os.listdir(img_dir)
    outliers = []
    for img_name in img_list:
        if img_name == 'Thumbs.db':
            continue
        img_path = os.path.normpath(os.path.join(img_dir, img_name))
        img = image.load_img(img_path, target_size=img_size) #os.readlink(img_path)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        pred = model.predict(x)
        pred = decode_predictions(pred, top=top_num)[0]
        if not is_cat_or_dog(pred):
            outliers.append(img_name)
    return outliers
    

In [37]:
outliers_xception = exclude_outliers_xception(top_num=top_num)
print(outliers_xception)
with open('./outliers/outliers_xception.txt','w') as f:
    f.write('\n'.join(outliers_xception))

['cat.7564.jpg', 'dog.2614.jpg', 'dog.4367.jpg']


### 利用inception_resnet_v2网络识别图片

In [38]:
def exclude_outliers_inception_resnet_v2(top_num=top_num, img_dir='train/' ):
    from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input, decode_predictions
    model = InceptionResNetV2(weights='imagenet')
    img_size = (299, 299)
    
    img_dir = os.path.normpath(os.path.join(dataset_path, img_dir))
    img_list = os.listdir(img_dir)
    outliers = []
    for img_name in img_list:
        if img_name == 'Thumbs.db':
            continue
        img_path = os.path.normpath(os.path.join(img_dir, img_name))
        img = image.load_img(img_path, target_size=img_size) #os.readlink(img_path)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        pred = model.predict(x)
        pred = decode_predictions(pred, top=top_num)[0]
        if not is_cat_or_dog(pred):
            outliers.append(img_name)
    return outliers
    

In [39]:
outliers_inception_resnet_v2 = exclude_outliers_inception_resnet_v2(top_num=top_num)
print(outliers_inception_resnet_v2)
with open('./outliers/outliers_inception_resnet_v2.txt','w') as f:
    f.write('\n'.join(outliers_inception_resnet_v2))

['cat.7564.jpg', 'dog.2614.jpg', 'dog.4367.jpg']


### 利用InceptionV3网络识别图片

In [40]:
def exclude_outliers_inception_v3(top_num=top_num, img_dir='train/' ):
    from keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
    model = InceptionV3(weights='imagenet')
    img_size = (299, 299)
    
    img_dir = os.path.normpath(os.path.join(dataset_path, img_dir))
    img_list = os.listdir(img_dir)
    outliers = []
    for img_name in img_list:
        if img_name == 'Thumbs.db':
            continue
        img_path = os.path.normpath(os.path.join(img_dir, img_name))
        img = image.load_img(img_path, target_size=img_size) #os.readlink(img_path)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        pred = model.predict(x)
        pred = decode_predictions(pred, top=top_num)[0]
        if not is_cat_or_dog(pred):
            outliers.append(img_name)
    return outliers
    

In [42]:
outliers_inception_v3 = exclude_outliers_inception_v3(top_num=top_num)
print(outliers_inception_v3)
with open('./outliers/outliers_inception_v3.txt','w') as f:
    f.write('\n'.join(outliers_inception_v3))

96116736/96112376 [==============================] - 88s 1us/step
['cat.7564.jpg', 'dog.2614.jpg', 'dog.4367.jpg']


### 求并集并将文件名称写入outliers.txt文件

In [43]:
outliers = list(set(outliers_xception+outliers_inception_resnet_v2+ outliers_inception_v3))
print(outliers)

outliers_file_path = './outliers/outliers.txt'

if os.path.isfile(outliers_file_path):
    os.remove(outliers_file_path)
with open(outliers_file_path, 'w') as f:
    f.write('\n'.join(outliers))


['dog.4367.jpg', 'cat.7564.jpg', 'dog.2614.jpg']


### 从outliers.txt文件中读取所识别的异常文件名称

In [8]:
outliers_file_path = './outliers_test.txt'
with open(outliers_file_path, 'r') as f:
    outliers_str = f.read()
    outliers = outliers_str.split('\n')
    print(outliers)

['dog.4367.jpg', 'cat.7564.jpg', 'dog.2614.jpg']


### 展示异常图片

In [51]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib._pylab_helpers import Gcf
from IPython.display import display_html
import base64
from ipykernel.pylab.backend_inline import show


def align_figures():

    images = []
    for figure_manager in Gcf.get_all_fig_managers():
        fig = figure_manager.canvas.figure
        png = get_ipython().display_formatter.format(fig)[0]['image/png']
        src = base64.encodebytes(png).decode()
        images.append('<img style="margin:0" align="left" src="data:image/png;base64,{}"/>'.format(src))

    html = "<div>{}</div>".format("".join(images))
    show._draw_called = False
    matplotlib.pyplot.close('all')
    display_html(html, raw=True)
    
def show_images(img_list, img_path, col_num=5):
    plt.figure(dpi=150)
    for i in range(len(img_list)):
        img = plt.imread(img_path+img_list[i])
        plt.subplot(len(img_list)//col_num+1, col_num, i+1)
        plt.title(img_list[i], fontsize=8)
        plt.axis('off')
        plt.imshow(img)
    align_figures()

In [53]:
show_num = 10
for i in range(len(outliers)//show_num+1):
    show_images(outliers[i*show_num:(i+1)*show_num], dataset_path+'train/',5)


## 将移除异常值的图像分成dog和cat文件夹

In [ ]:
outliers_file_path = './outliers_test.txt'
with open(outliers_file_path, 'r') as f:
    outliers_str = f.read()
    outliers = outliers_str.split('\n')
    print(outliers)

### 创建dog和cat分类文件夹

In [19]:
#  创建train_sep/dog及train_sep/cat文件夹
dog_dir = os.path.normpath(os.path.join(dataset_path, './train_sep/dog'))
if not os.path.exists(dog_dir):
    os.makedirs(dog_dir)
    print('Directory "'+dog_dir+'" created!')
else:
    print('Directory "'+dog_dir+'" exists!')


cat_dir = os.path.normpath(os.path.join(dataset_path, './train_sep/cat'))
if not os.path.exists(cat_dir):
    os.makedirs(cat_dir)
    print('Directory "'+cat_dir+'" created!')
else:
    print('Directory "'+cat_dir+'" exists!')


Directory "test_data\train_sep\dog" created!
Directory "test_data\train_sep\cat" created!


### 复制图片到文件夹

In [21]:
from sklearn.model_selection import train_test_split

# 验证集比率
val_fraction = 0.2

# 对训练文件名进行分类
train_filenames = os.listdir(os.path.normpath(os.path.join(dataset_path, 'train')))
cat_filenames = filter(lambda x: x[:3] == 'cat', train_filenames)
dog_filenames = filter(lambda x: x[:3] == 'dog', train_filenames)

# 删除异常值
cat_filenames = np.array([filename for filename in cat_filenames if filename not in outliers])
dog_filenames = np.array([filename for filename in dog_filenames if filename not in outliers])

print('Number of Cat files:', len(cat_filenames))
print('Number of Dog files:', len(dog_filenames))

def copyFiles(category, names):
    from shutil import copyfile
    path = os.path.normpath( os.path.join(dataset_path ,'train_sep/', category))
    if len(os.listdir(path)) <=0 :
        for filename in names:
            target_file_path = os.path.normpath( os.path.join(path, filename) )
            origin_file_path = os.path.normpath( os.path.join(dataset_path, './train/', filename) )
            if not os.path.isfile(target_file_path):
                copyfile(origin_file_path, target_file_path)
        print('train '+ category +' copyfile success')
    else:
        print('train '+ category +' files exists')

copyFiles('cat', cat_filenames)
copyFiles('dog', dog_filenames)

Number of Cat files: 13
Number of Dog files: 7
train cat files exists
train dog copyfile success


> 根据上面的图片，手动排除正确的猫狗（也就是识别错误的），将真正的异常值写入`real_outliers.txt`文件